# Deeper Networks for Image Classification

## VGG16 Model for MNIST Image Classification

- Code by: Kaviraj Gosaye
- Student ID: 220575371

In [ ]:
# GPU details
output = !nvidia-smi
for line in output:
    print(line)
    if line.strip() == "":
        break

In [ ]:
import psutil
# Get virtual memory information
virtual_memory = psutil.virtual_memory()
v_mem = virtual_memory.available / (1024 ** 3)

# Print available virtual memory
print("Available virtual memory:", v_mem)

### 0. Imports

In [ ]:
# import libraries
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
from torchinfo import summary
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score
import seaborn as sns
from datetime import datetime

### 1. Data Loading and Preprocessing

In [ ]:
# transform PIL image to tensor and normalize
transform = transforms.Compose([
    transforms.ToTensor(), 
    transforms.Normalize((0.5), (0.5))])

# load mnist dataset
train_set = torchvision.datasets.MNIST(root='../datasets', train=True, download=True, transform=transform)
test_set = torchvision.datasets.MNIST(root='../datasets', train=False, download=True, transform=transform)
                                    
train_loader = torch.utils.data.DataLoader(train_set, batch_size=64, shuffle=True, num_workers=8)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=64, shuffle=False, num_workers=8)

In [ ]:
# extract one sample from the training set
dataiter = iter(train_loader)
# images, labels = dataiter.next()
images, labels = next(dataiter)

# plot the image
def imshow(img):
    # reverse normalization
    img = img / 2 + 0.5
    # convert tensor to numpy array
    npimg = img.numpy()
    # rearrange the dimensions to match matplotlib format
    # matplotlib:   H x W x C
    # torch:        C x H x W
    plt.imshow(np.transpose(npimg, (1, 2, 0)))

# show images
imshow(torchvision.utils.make_grid(images))

### 2. Model Building

In [ ]:
# defining the VGG16 layers
VGG16_layers = [64, 64, "M", 128, 128, "M", 256, 256, 256, "A", 512, 512, 512, "A", 512, 512, 512, "M" ]

In [ ]:
# creating a class of the VGG16 model
class VGG16(nn.Module):
    def __init__(self, in_channels=3, num_classes=1000):
        super(VGG16, self).__init__()
        # inpuy layer
        self.in_channels = in_channels
        # hidden layers
        self.hidden_layers = self.conv_layers(VGG16_layers)
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
        # output layer
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.hidden_layers(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

    # function to create the hidden convolutional layers
    def conv_layers(self, layer_types):
        layers = []
        in_channels = self.in_channels

        for layer in layer_types:
            if type(layer) == int:
                out_channels = layer
                layers += [ nn.Conv2d( in_channels=in_channels, out_channels=out_channels, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), ), nn.BatchNorm2d(layer), nn.ReLU()]
                in_channels = layer
            # max pooling layer
            elif layer == "M":
                layers += [nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))]
            # average pooling layer
            elif layer == "A":
                layers += [nn.AdaptiveAvgPool2d((14,14))]

        return nn.Sequential(*layers)

In [ ]:
# setting the device to cuda if available
device = "cuda" if torch.cuda.is_available() else "cpu"

# creating instance of model and setting it to the device
vgg16 = VGG16(in_channels=1, num_classes=10).to(device)

In [ ]:
# visualize the model
info = summary(vgg16, (1, 1, 28, 28), col_names = ('input_size', 'output_size', 'num_params', 'kernel_size'))
print(info)

### 3. Model Training

In [ ]:
# loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(vgg16.parameters(), lr=0.001)

# open log file in write mode
log_file = open("../logs/vgg16_mnist_log.txt", "w")

# training the model
start = time.time()

num_epochs = 10
losses = []
train_accs = []

for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()

        outputs = vgg16(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # cumulative loss
        running_loss += loss.item()
        # printing the average loss every 100 mini-batches
        if i % 100 == 99:
            # Get current timestamp
            timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            # Write loss to log file
            log_file.write(f"{timestamp} - [{epoch + 1}, {i + 1}] loss: {running_loss / 100}\n")
            print(f"{timestamp} - [{epoch + 1}, {i + 1}] loss: {running_loss / 100}")
            running_loss = 0.0

    correct = 0.0
    pred = outputs.argmax(dim=1, keepdim=True)
    # reshaping the labels to match the shape of the predictions
    # comparing the predictions to the labels using element-wise comparison
    # summing the correct predictions
    correct += pred.eq(labels.view_as(pred)).sum().item()
    train_acc = 100. * correct / len(outputs)
    train_accs.append(train_acc)
    losses.append(loss.item())
    
print(f"Finished Training after {time.time() - start} seconds")

# Close the log file
log_file.close()

In [ ]:
# save model after training
torch.save(vgg16, "../Models/vgg16_mnist.pth")

### 4. Model Evaluation

In [ ]:
vgg16.to(device)
vgg16.eval()
# Disable gradient calculation
with torch.no_grad():
    correct = 0
    total = 0
    predicted_labels = []
    true_labels = []
    test_loss = 0

    # Using test set
    for data in test_loader:
        images, labels = data[0].to(device), data[1].to(device)
        
        # Forward pass
        outputs = vgg16(images)
        
        # Calculate the test loss
        loss = criterion(outputs, labels)
        test_loss += loss.item()

        # Get the predicted labels
        _, predicted = torch.max(outputs.data, 1)
        
        # Update the total and correct predictions
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # Append the predicted and true labels
        predicted_labels.extend(predicted.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

    # Calculate the accuracy
    accuracy = 100 * correct / total
    test_loss /= len(test_loader.dataset)

# Print the accuracy and test loss
print(f"Accuracy on the test data: {accuracy}%")
print(f"Test Loss: {test_loss}")

# Create the confusion matrix
cm = confusion_matrix(true_labels, predicted_labels)

# Plot the confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.title("Confusion Matrix - VGG16 on MNIST Dataset")
plt.show()

In [ ]:
# Plot the loss
plt.plot(losses)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss of VGG16 on MNIST Dataset')
plt.show()

In [ ]:
# Plot the accuracy
plt.plot(train_accs)
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training Accuracy of VGG16 on MNIST Dataset')
plt.show()

In [ ]:
# Convert lists to tensors
pred_labels_tensor = torch.tensor(predicted_labels)
true_labels_tensor = torch.tensor(true_labels)

# Calculate precision and recall
precision = precision_score(true_labels_tensor, pred_labels_tensor, average='weighted')
recall = recall_score(true_labels_tensor, pred_labels_tensor, average='weighted')

print("Precision:", precision)
print("Recall:", recall)

### 5. Model Testing

In [ ]:
# loading some images from the test set
testloader = torch.utils.data.DataLoader(test_set, batch_size=30,shuffle=False)

dataiter = iter(testloader)
images, true_labels = next(dataiter)

In [ ]:
# classes of mnist
classes = ('0', '1', '2', '3', '4', '5', '6', '7', '8', '9')

In [ ]:
# plotter function to plot the images
def plotter(images, true_labels, predicted_labels):
    correct = 0
    num_images = len(images)
    num_rows = (num_images) // 5
    fig, axs = plt.subplots(num_rows, 5, figsize=(15, 20))
    # setting the title
    fig.suptitle('True Label - [Predicted Labels]', fontsize=20)

    for ind in range(num_images):
        # plot image
        img = images[ind].permute(1, 2, 0) / 2 + 0.5
        ax = axs[ind // 5, ind % 5]
        ax.imshow(img, cmap='gray')

        # left - true label, right - predicted label
        if classes[true_labels[ind]] == classes[predicted_labels[ind]]:
            color = 'green'
            title = classes[true_labels[ind]] + ' - [' + classes[predicted_labels[ind]] + ']'
            ax.set_title(title, color=color)
            correct += 1
        else:
            color = 'red'
            title = classes[true_labels[ind]] + ' - [' + classes[predicted_labels[ind]] + ']'
            ax.set_title(title, color=color)        

    plt.tight_layout(pad=2.0)
    plt.show()
    print('Number of correct predictions: ', correct)
    print('Number of wrong predictions: ', num_images - correct)

In [ ]:
# use model to predict images label
vgg16.eval()
vgg16.to('cpu')
outputs = vgg16(images)
_, predicted_labels = torch.max(outputs, 1)

In [ ]:
# plot the images and predictions
plotter(images, true_labels, predicted_labels)